In [1]:
import torch

import pathpyG as pp

print('Running on', pp.config['torch']['device'])

Running on cuda


The following snippet generates a graph with three nodes a, b, c and three edges (a,c), (b,c), (a,b)

In [2]:
g = pp.Graph(edge_index=torch.tensor([[0,1,0], [2,2,1]]).to(pp.config['torch']['device']), node_id=['a', 'b', 'c'])
print(g)

Graph with 3 nodes and 3 edges

Node attributes
	node_id		<class 'list'>

Graph attributes
	num_nodes		<class 'int'>



In [3]:
g.get_sparse_adj_matrix().getrow(0)

<1x3 sparse matrix of type '<class 'numpy.float32'>'
	with 2 stored elements in Compressed Sparse Row format>

In [4]:
for v in g.nodes:
    print(v)

for e in g.edges:
    print(e)

a
b
c
('a', 'c')
('b', 'c')
('a', 'b')


In [14]:
g.in_degrees['b']

1

In [15]:
g.out_degrees['a']

2

In [16]:
'b' in g.successors('c')

False

In [17]:
g.is_edge('a', 'b')

True

We can create the same graph based on an edge list.

In [9]:
g = pp.Graph.from_edge_list([['a','b'], ['b','c'], ['a','c']])
print(g)
print(g.data.edge_index)
print(g.node_index_to_id)
print(g.node_id_to_index)

Graph with 3 nodes and 3 edges

Node attributes
	node_id		<class 'list'>

Graph attributes
	num_nodes		<class 'int'>

tensor([[0, 1, 0],
        [1, 2, 2]], device='cuda:0')
{0: 'a', 1: 'b', 2: 'c'}
{'a': 0, 'b': 1, 'c': 2}


In [10]:
g.data['node_class'] = torch.tensor([[1],[2],[3]]).to(pp.config['torch']['device'])
g.data['edge_weight'] = torch.tensor([[1],[1],[2]]).to(pp.config['torch']['device'])
g.data['graph_feature'] = torch.tensor([42]).to(pp.config['torch']['device'])
print(g)

Graph with 3 nodes and 3 edges

Node attributes
	node_class		<class 'torch.Tensor'> -> torch.Size([3, 1])
	node_id		<class 'list'>

Edge attributes
	edge_weight		<class 'torch.Tensor'> -> torch.Size([3, 1])

Graph attributes
	graph_feature		<class 'torch.Tensor'> -> torch.Size([1])
	num_nodes		<class 'int'>



We can conveniently access attributes of individual nodes and edges as follows:

In [11]:
print(g['node_class', 'b'])
print(g['edge_weight', 'a', 'c'])
print(g['graph_feature'])

tensor([2], device='cuda:0')
tensor([2], device='cuda:0')
tensor([42], device='cuda:0')


We can iterate through successors and predecessors of nodes:

In [12]:
for w in g.successors('a'):
    print(w)
print('---')
for w in g.predecessors('b'):
    print(w)

b
c
---
a


We can compute weighted adjacency matrices as follows:

In [13]:
print(g.get_sparse_adj_matrix(edge_attr='edge_weight').todense())

[[0 1 2]
 [0 0 1]
 [0 0 0]]


To easily apply GNNs, we can add attributes based on one-hot-encodings of nodes and edges:

In [14]:
g.add_node_ohe(attr_name='node_feature_1')
g.add_node_ohe(attr_name='node_feature_2', dim=4)
g.add_edge_ohe(attr_name='edge_feature', dim=5)
print(g)

print(g.data['node_feature_1'])
print(g.data['node_feature_2'])
print(g.data['edge_feature'])

Graph with 3 nodes and 3 edges

Node attributes
	node_class		<class 'torch.Tensor'> -> torch.Size([3, 1])
	node_feature_2		<class 'torch.Tensor'> -> torch.Size([3, 4])
	node_id		<class 'list'>
	node_feature_1		<class 'torch.Tensor'> -> torch.Size([3, 3])

Edge attributes
	edge_feature		<class 'torch.Tensor'> -> torch.Size([3, 5])
	edge_weight		<class 'torch.Tensor'> -> torch.Size([3, 1])

Graph attributes
	graph_feature		<class 'torch.Tensor'> -> torch.Size([1])
	num_nodes		<class 'int'>

tensor([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]], device='cuda:0')
tensor([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.]], device='cuda:0')
tensor([[1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0.]], device='cuda:0')


We can generate pyG data frames from a Graph. We can easily combine multiple node or edge features which have a tensor value.

In [15]:
data = g.to_pyg_data()
print(data)

Data(edge_index=[2, 3], node_id=[3], num_nodes=3, node_class=[3, 1], edge_weight=[3, 1], graph_feature=[1], node_feature_1=[3, 3], node_feature_2=[3, 4], edge_feature=[3, 5])


Example for a graph with multi-edges:

In [16]:
g = pp.Graph.from_edge_list([['a','b'], ['b','c'], ['a','c'], ['a', 'b']])
print(g)
print(g.data.edge_index)
print(g.node_index_to_id)
print(g.node_id_to_index)

Graph with 3 nodes and 4 edges

Node attributes
	node_id		<class 'list'>

Graph attributes
	num_nodes		<class 'int'>

tensor([[0, 1, 0, 0],
        [1, 2, 2, 1]], device='cuda:0')
{0: 'a', 1: 'b', 2: 'c'}
{'a': 0, 'b': 1, 'c': 2}


In [17]:
for e in g.edges:
    print(e)

('a', 'b')
('b', 'c')
('a', 'c')
('a', 'b')


Generate a graph from a pyG data set: 

In [18]:
from torch_geometric.data import Data
d = Data(x=torch.Tensor([0,0,0,1]), edge_index=torch.Tensor([[0,0,1],[1,2,2]]), node_id=['a', 'b', 'c', 'd'])
d.to(pp.config['torch']['device'])

Data(x=[4], edge_index=[2, 3], node_id=[4])

In [19]:
print(d)

Data(x=[4], edge_index=[2, 3], node_id=[4])


In [20]:
g = pp.Graph.from_pyg_data(d)
print(g)

Graph with 4 nodes and 3 edges

Node attributes
	node_id		<class 'list'>
	x		<class 'torch.Tensor'> -> torch.Size([4])

Graph attributes
	num_nodes		<class 'int'>

